In [2]:
# Feed image for image detection
import cv2

In [8]:
cascPath = "./haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

In [12]:
# path to cropped face images
path='./tmp/dataset/'

In [3]:
import os
import glob

In [21]:
# Get Image names stored in "Images" folder
image_path_names=[]
person_names=set()
for folder_name in glob.glob(path + 'train/*'):
    person_names.add(folder_name.split('/')[-1])
    for file_name in glob.glob(folder_name + '/*'):
        image_path_names.append(file_name)

In [22]:
len(image_path_names)

343

In [23]:
person_names

{'aman', 'ayush', 'naman'}

In [0]:
os.mkdir(path+'test/')

In [27]:
# Create Separate folder for each person in "Test_Images_crop" folder
for person in person_names:
  os.mkdir(path+'/test/'+person+'/')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

2022-06-06 19:09:56.570350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cert/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-06-06 19:09:56.570459: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [16]:
#Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

2022-06-06 19:11:35.554505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cert/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-06-06 19:11:35.554614: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-06 19:11:35.554693: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nj): /proc/driver/nvidia/version does not exist
2022-06-06 19:11:35.555710: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-06 19:11:36.249437: W tensorflo

In [38]:
def load_weights_by_name(model, path, verbose=False):
    import h5py
    def load_model_weights(cmodel, weights):
        for layer in cmodel.layers:
            print(layer.name)
            if hasattr(layer, 'layers'):
                load_model_weights(layer, weights[layer.name])
            else:
                for w in layer.weights:
                    _, name = w.name.split('/')
                    if verbose:
                        print(w.name)
                    try:
                        w.assign(weights[layer.name][name][()])
                    except:
                        w.assign(weights[layer.name][layer.name][name][()])

    with h5py.File(path, 'r') as f:
        load_model_weights(model, f)

In [32]:
# load_weights_by_name(model, './vgg-face-keras.h5', True)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     7

In [18]:
# Remove Last Softmax layer and get model upto last flatten layer with outputs 2622 units
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [45]:
#Prepare Training Data
x_train=[]
y_train=[]
person_folders=os.listdir(path+'/train/')
person_rep=dict()
for i,person in enumerate(person_folders):
    person_rep[i]=person
    image_names=os.listdir(path + 'train/'+person+'/')
    for image_name in image_names:
        img=load_img(path+'/train/'+person+'/'+image_name,target_size=(224,224))
        img=img_to_array(img)
        img=np.expand_dims(img,axis=0)
        img=preprocess_input(img)
        img_encode=vgg_face(img)
        x_train.append(np.squeeze(K.eval(img_encode)).tolist())
        y_train.append(i)

In [46]:
person_rep

{0: 'ayush', 1: 'aman', 2: 'naman'}

In [47]:
x_train=np.array(x_train)
y_train=np.array(y_train)

In [54]:
#Prepare Test Data
x_test=[]
y_test=[]
person_folders=os.listdir(path+'/test/')
for i,person in enumerate(person_folders):
    image_names=os.listdir(path+'test/'+person+'/')
    for image_name in image_names:
        img=load_img(path+'/test/'+person+'/'+image_name,target_size=(224,224))
        img=img_to_array(img)
        img=np.expand_dims(img,axis=0)
        img=preprocess_input(img)
        img_encode=vgg_face(img)
        x_test.append(np.squeeze(K.eval(img_encode)).tolist())
        y_test.append(i)

In [55]:
x_test=np.array(x_test)
y_test=np.array(y_test)

In [56]:
# Save test and train data for later use
np.save('train_data',x_train)
np.save('train_labels',y_train)
np.save('test_data',x_test)
np.save('test_labels',y_test)

In [57]:
# Load saved data
x_train=np.load('train_data.npy')
y_train=np.load('train_labels.npy')
x_test=np.load('test_data.npy')
y_test=np.load('test_labels.npy')

In [49]:
# Softmax regressor to classify images based on encoding 
classifier_model=Sequential()
classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.3))
classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=6,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='nadam',metrics=['accuracy'])

In [58]:
classifier_model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

2022-06-06 10:22:44.265760: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
11/11 [==============================] - 29s 81ms/step - loss: 0.7705 - accuracy: 0.8192 - val_loss: 1.5891 - val_accuracy: 0.3333
Epoch 2/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4221 - accuracy: 0.9738 - val_loss: 1.5667 - val_accuracy: 0.3333
Epoch 3/100
11/11 [==============================] - 0s 7ms/step - loss: 0.3658 - accuracy: 0.9825 - val_loss: 1.5352 - val_accuracy: 0.3333
Epoch 4/100
11/11 [==============================] - 0s 7ms/step - loss: 0.3293 - accuracy: 0.9796 - val_loss: 1.5155 - val_accuracy: 0.3333
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.2955 - accuracy: 1.0000 - val_loss: 1.4744 - val_accuracy: 0.3333
Epoch 6/100
11/11 [==============================] - 0s 7ms/step - loss: 0.2669 - accuracy: 1.0000 - val_loss: 1.4366 - val_accuracy: 0.4444
Epoch 7/100
11/11 [==============================] - 0s 7ms/step - loss: 0.2454 - accuracy: 0.9971 - val_loss: 1.3715 - val_accuracy: 0.6667
Epoch 8/10

11/11 [==============================] - 0s 8ms/step - loss: 0.0207 - accuracy: 1.0000 - val_loss: 0.0160 - val_accuracy: 1.0000
Epoch 59/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0269 - accuracy: 1.0000 - val_loss: 0.0271 - val_accuracy: 1.0000
Epoch 60/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0241 - accuracy: 1.0000 - val_loss: 0.0209 - val_accuracy: 1.0000
Epoch 61/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0264 - accuracy: 1.0000 - val_loss: 0.1812 - val_accuracy: 1.0000
Epoch 62/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0424 - val_accuracy: 1.0000
Epoch 63/100
11/11 [==============================] - 0s 10ms/step - loss: 0.0294 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 1.0000
Epoch 64/100
11/11 [==============================] - 0s 10ms/step - loss: 0.0285 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 1.0000
Epoch 65/100
11/1

In [59]:
# Save model for later use
tf.keras.models.save_model(classifier_model,'face_classifier_model.h5')

In [23]:
# Load saved model
classifier_model=tf.keras.models.load_model('face_classifier_model.h5')

In [22]:
import matplotlib.pyplot as plt

In [71]:
def plot(img):
  plt.figure(figsize=(8,4))
  plt.imshow(img[:,:,::-1])
  plt.show()

In [27]:
# Label names for class numbers
person_rep={0: 'ayush', 1: 'aman', 2: 'naman'}

In [61]:
os.mkdir(path+'/Predictions')

In [65]:
os.mkdir(path+'/Test_Images')

In [33]:
for img_name in os.listdir(path+'Test_Images/'):
    print(img_name)
    if img_name=='crop_img.jpg':
        continue
    
    # Load Image
    img=cv2.imread(path+'/Test_Images/'+img_name)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Faces
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    left,top,right,bottom=0,0,0,0
    for (i, face) in enumerate(faces):
        # Extract Each Face
        x, y, w, h = face
        print(face)
        img_crop=img[y:y+h,x:x+w]
        cv2.imwrite(path+'/Test_Images/crop_img.jpg',img_crop)

        # Get Embeddings
        crop_img=load_img(path+'/Test_Images/crop_img.jpg',target_size=(224,224))
        crop_img=img_to_array(crop_img)
        crop_img=np.expand_dims(crop_img,axis=0)
        crop_img=preprocess_input(crop_img)
        img_encode=vgg_face(crop_img)

        # Make Predictions
        embed  = K.eval(img_encode)
        person = classifier_model.predict(embed)
        name   = person_rep[np.argmax(person)]
        os.remove(path + '/Test_Images/crop_img.jpg')
        
        cv2.rectangle(img,(x, y),(x+w,y+h),(0, 255, 0), 2)
        img=cv2.putText(img,name,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
        img=cv2.putText(img,str(np.max(person)),(x,y+h),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,0),1,cv2.LINE_AA)
      
    # Save images with bounding box,name and accuracy 
    cv2.imwrite(path+'/Predictions/'+img_name,img)
    plot(img)

In [ ]:
# Webcam

In [28]:
cap = cv2.VideoCapture(0)

In [34]:
# while True:
#     success, img = cap.read()

#     faces = faceCascade.detectMultiScale(img,
#                                          scaleFactor=1.1,
#                                          minNeighbors=5,
#                                          minSize=(60, 60),
#                                          flags=cv2.CASCADE_SCALE_IMAGE)
    
#     left,top,right,bottom=0,0,0,0
#     for (i, face) in enumerate(faces):
#         # Extract Each Face
#         x, y, w, h = face
#         print(face)
#         img_crop=img[y:y+h,x:x+w]
#         cv2.imwrite(path+'/Test_Images/crop_img.jpg',img_crop)

#         # Get Embeddings
#         crop_img=load_img(path+'/Test_Images/crop_img.jpg',target_size=(224,224))
#         crop_img=img_to_array(crop_img)
#         crop_img=np.expand_dims(crop_img,axis=0)
#         crop_img=preprocess_input(crop_img)
#         img_encode=vgg_face(crop_img)

#         # Make Predictions
#         embed  = K.eval(img_encode)
#         person = classifier_model.predict(embed)
#         name   = person_rep[np.argmax(person)]
#         os.remove(path + '/Test_Images/crop_img.jpg')
        
#         cv2.rectangle(img,(x, y),(x+w,y+h),(0, 255, 0), 2)
#         img=cv2.putText(img,name,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
#         img=cv2.putText(img,str(np.max(person)),(x,y+h),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,0),1,cv2.LINE_AA)
      
#     # Save images with bounding box,name and accuracy 
#     if img is not None:
#         cv2.imshow("Frame", img)
    
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

In [30]:
cap.release()
cv2.destroyAllWindows()